<a href="https://colab.research.google.com/github/srihari-humbarwadi/Real-or-Not-NLP-with-Disaster-Tweets/blob/master/tpu_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense
from transformers import TFBertModel
import pandas as pd
from transformers.tokenization_bert import BertTokenizer
from sklearn.model_selection import train_test_split

print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-rc1


In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.105.93.194:8470']
INFO:tensorflow:Initializing the TPU system: 10.105.93.194:8470


INFO:tensorflow:Initializing the TPU system: 10.105.93.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [0]:
batch_size = 8 * strategy.num_replicas_in_sync
epochs = 25

autotune = tf.data.experimental.AUTOTUNE
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

In [0]:
 with strategy.scope():
    tweets = tf.keras.Input(shape=(512,), dtype=tf.int32)
    bert = TFBertModel.from_pretrained(pretrained_weights)
    tweets_hidden_mean = tf.reduce_mean(bert(tweets)[0], axis=1)
    x = Dense(units=64)(tweets_hidden_mean)
    logits = Dense(units=1, name='logits', activation='sigmoid')(x)
    model = tf.keras.Model(inputs=[tweets], outputs=[logits])

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 512, 768), (None, 109482240 
_________________________________________________________________
tf_op_layer_Mean (TensorFlow [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                49216     
_________________________________________________________________
logits (Dense)               (None, 1)                 65        
Total params: 109,531,521
Trainable params: 109,531,521
Non-trainable params: 0
_________________________________________________________________


In [7]:
df = pd.read_csv('train.csv')

encoded_text = [tokenizer.encode(text, max_length=512, pad_to_max_length=True) for text in df['text']]
labels = df['target']
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
x_train, x_test, y_train, y_test = train_test_split(encoded_text, labels, test_size=0.15)
len(x_train), len(x_test)

(6471, 1142)

In [0]:
def make_tfdataset(x, y):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.shuffle(512)
    dataset = dataset.repeat(epochs)
    dataset = dataset.prefetch(autotune)
    return dataset

In [0]:
with strategy.scope():
    train_dataset = make_tfdataset(x_train, y_train)
    test_dataset = make_tfdataset(x_test, y_test)

In [0]:
with strategy.scope():
    metrics_list = [tf.metrics.BinaryAccuracy(), tf.metrics.Precision(), tf.metrics.Recall()]
    model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=False),
                  metrics=metrics_list,
                  optimizer=tf.optimizers.Adam(1e-4))
model.fit(train_dataset, epochs=epochs, validation_data=test_dataset)    

Train for 2525 steps, validate for 425 steps
Epoch 1/25


 555/2525 [=====>........................] - ETA: 12:00 - loss: 0.4706 - binary_accuracy: 0.7713 - precision: 0.7801 - recall: 0.6532